In [ ]:
# Snippet extracter. 
# This tool is the snippet extracter. A python JSON/CSV parser that, item by item:
# 	- extracts the full-text
# 	- identifies and counts the number of definitional phrases in the full-text
# 	- identifies the surrounding (pre, post, and including) sentences for each (the definitonal snippet)
# 	- if only one phrase, writes the definitional snippet to that entry
# 	- if more than one phrase, creates a new entry for every occurrence above 1.

In [43]:
# FILE LOADER

from pathlib import Path, PureWindowsPath

# First, try to load the JSON file in downloads.

folder_name = Path("C:/Users/Rock/Downloads")
json_file = folder_name  / '250_zot2json.json'

print(json_file)



C:\Users\Rock\Downloads\250_zot2json.json


In [6]:
# GET THE TITLES ONLY

# prefix=type, event=string, value=FeatureCollection


import ijson


def extract_lot_numbers(json_filename):
    with open(json_filename, 'rb') as input_file:
        print('opened')
        the_titles = ijson.items(input_file, 'item.title')
        print(the_titles)
        for titl in the_titles:
            print('Title: ' + titl)


if __name__ == '__main__':
    extract_lot_numbers(json_file)

# However, I think this is NOT what I want to do, because it loads all the items in a list.
# That would mean all the fulltext would be in one list, and that would be a ~50 MB list.
# Instead, I should probably use the parse function (https://pypi.org/project/ijson/#id10)
# to do something to each event I encounter. This way I can deal with one fulltext result
# at a time, split and parse the definitional snippet, append to an array or CSV, and then
# move on to the next fulltext, never having to keep more than one in memory.


opened
Lot number: Political Compassions under Pandemic Spectacles
Lot number: Literature-based discovery for oceanographic climate science
Lot number: SCHEMA-an algorithm for automated product taxonomy mapping in e-commerce
Lot number: The integration of maintenance plans and production scheduling taking account of outsourcing: a literature review
Lot number: Us and them: Black-White relations in the wake of Hispanic population growth
Lot number: Sorcery and Spiritual Hegemony in Africa
Lot number: Visual structures for generative design search spaces
Lot number: OPTIMIZING RESEARCH AND DEVELOPMENT PERFORMANCE USING KNOWLEDGE MANAGEMENT
Lot number: Collecting and Collected: Native American Subjectivity and Transatlantic Transactions in The Female American
Lot number: Transaction Routing and its Verification by Correct Model Transformations
Lot number: A RESQML Virtual Cloud Storage and Access Via (Api) In the Oil and Gas Industry
Lot number: CONQUEST AND COSMOLOGIES
Lot number: Data d

In [ ]:
# FULLTEXT LENGTH CALCULATOR
# First test. Go through and calculate the length of each fulltext.
# Put each number into an array and then print the array.

import ijson


lenarray = []

with open(json_file, 'rb') as input_file:
    parser = ijson.parse(input_file)
    for prefix, event, value in parser:
        # print(prefix, event)
        # if (prefix) == ('title'):
        #     print('in the loop')
        #     thelen = len(value)
        #     lenarray.append(thelen)

        if (prefix, event) == ('item.fulltext', 'string'):
            # print('in the loop')
            snip = value[:5]
            thelen = len(value)
            lenarray.append((thelen, snip))

print(lenarray)






In [138]:
# SNIPPET-MAKER CODE

# TODO: I may need to replace the i without the period with i.
# That could also be messing up define or hierarchy.

# Also, I can do wrapping if I need to:
# lines = tr.wrap(my_str, width=30) # text is wrapped before it hits 30.


import ijson
import re
import textwrap as tr

list_of_phrases = ["define hierarchy",
"defines hierarchy",
"defining hierarchy",
"definition for hierarchy",
"definition hierarchy",
"definition of hierarchy",
"hierarchy defined",
"hierarchy definition",
"hierarchy meaning",
"hierarchy refers",
"hierarchy they mean",
"hierarchy we mean",
"hierarchy i mean",
"define hierarchical",
"defines hierarchical",
"definition for hierarchical",
"definition hierarchical",
"definition of hierarchical",
"hierarchical defined",
"hierarchical definition",
"hierarchical i mean",
"hierarchical meaning",
"hierarchical they mean",
"hierarchical we mean",
"hierarchical refers"]

review_array = []

# How many characters on either side of the search phrase.
num_buffer = 280

# This function uses the lowercase for search, but then snips
# from fulltext that hasn't been made all lowercase so that the
# final snippets are more readable.

def get_snippets(sphrase, fulltextstr, lower_fulltextstr, fulltext_len):
    snip_array = []
    for occurrence in re.finditer(sphrase, lower_fulltextstr):
        if occurrence.start() != -1: # we don't have an else here, but could
            snip_start = occurrence.start() - num_buffer
            snip_end = occurrence.end() + num_buffer
            # print(sphrase, snip_start, snip_end)
            if snip_start < 0:
                snip_start = 0
            if snip_end > fulltext_len:
                snip_end = fulltext_len
            snippet = fulltextstr[snip_start:snip_end]
            # Make search phrase all caps to be easier to spot. Can be commented out.
            snippet = snippet.replace(sphrase, sphrase.upper()) 
            # print(snippet)
            snip_array.append(snippet)
    return(snip_array)

with open(json_file, 'rb') as input_file:
    parser = ijson.parse(input_file)

    item_key = ''
    item_title = ''

    for prefix, event, value in parser:
        # TODO:
        # I'll need to get the key and titles here with ifs.
        # The value is different json info depending on the prefix.
         
        if (prefix, event) == ('item.key', 'string'):
            item_key = value # The Zotero key should be a unique ID

        if (prefix, event) == ('item.title', 'string'):
            item_title = value

        if (prefix, event) == ('item.fulltext', 'string'):
            # print('in the loop')
            nofi_fulltextstr = value.replace('ﬁ','fi')
            # clean_value = nofi_value
            # clean_value = re.sub(r'[_\W]+', ' ', value)
            lower_fulltextstr = nofi_fulltextstr.lower()
            # clean_value = re.sub(r'[_\W]+', ' ', value).lower().strip().replace('ﬁ','fi')
            # re.sub(r'[_\W]+', ' ', value).strip().lower()
            # if thelen == 107811: for testing
            fulltext_len = len(lower_fulltextstr)
            termcounter = 0
            for phrase in list_of_phrases:                
                if phrase in lower_fulltextstr:
                    termcounter = termcounter + 1
                    # print((thelen, phrase))
                    snippet_array = get_snippets(phrase, nofi_fulltextstr, lower_fulltextstr, fulltext_len)
                    for snip in snippet_array:
                        review_array.append([item_key, item_title, phrase, snip])
                    # if thelen == 107811:
                    #     print(value)
                    #     print(" +++++++++++++++++++ STARTING STRIPPED STRING NOW +++++++++++++++++++++++++")
                    #     print(re.sub(r'[_\W]+', ' ', value).strip().lower())
            # if termcounter == 0: # and thelen > 400000 and thelen < 600000:                
            #     print("=========================== NEW ENTRY =================================")
            #     print(value)
                # print(" +++++++++++++++++++ STARTING STRIPPED STRING NOW +++++++++++++++++++++++++")
                # print(re.sub(r'[_\W]+', ' ', value).strip().lower())


# I can write this as a JSON file. Seemingly it's easy, one command.
# https://stackoverflow.com/a/56981871

print(len(review_array))


200


In [14]:
# Now I would like to make a dataframe.
# Use that dataframe to make a tab-separated CSV.
# Try opening that with excel and see how well it works.

# Then I need to iterate through all _zot2json.json files
# to create the snippet files

from pathlib import Path, PureWindowsPath
import os

# Example of loading the JSON file in downloads.
# folder_name = Path("C:/Users/Rock/Downloads")
# json_file = folder_name  / '250_zot2json.json'

# Make list of all files matching the name.

json_folder = Path("P:/Documents/research/1_sqlr_hierarchy/data/zotero_json_fulltext")
folder = json_folder.glob('**/*zot2json.json')
print(type(folder))
zot2json_files = [x for x in folder if x.is_file()]
print(type(zot2json_files))
print(type(zot2json_files[0]))
print(zot2json_files)

# We've now got the list we need to loop over:
# zot2json_files
# for jfile in zot2json_files:
# with jfile.open() as inputf:

# File parsing test code

for i in zot2json_files:
    foo = str(i)
    print(foo)
    the_end = foo.split(sep='\\')[-1]
    print(the_end)
    the_prefix = the_end.split(sep='_')[0]
    print('The prefix:', the_prefix)




<class 'generator'>
<class 'list'>
<class 'pathlib.WindowsPath'>
[WindowsPath('P:/Documents/research/1_sqlr_hierarchy/data/zotero_json_fulltext/6857_zot2json.json'), WindowsPath('P:/Documents/research/1_sqlr_hierarchy/data/zotero_json_fulltext/6750_zot2json.json'), WindowsPath('P:/Documents/research/1_sqlr_hierarchy/data/zotero_json_fulltext/6500_zot2json.json'), WindowsPath('P:/Documents/research/1_sqlr_hierarchy/data/zotero_json_fulltext/6250_zot2json.json'), WindowsPath('P:/Documents/research/1_sqlr_hierarchy/data/zotero_json_fulltext/6000_zot2json.json'), WindowsPath('P:/Documents/research/1_sqlr_hierarchy/data/zotero_json_fulltext/5750_zot2json.json'), WindowsPath('P:/Documents/research/1_sqlr_hierarchy/data/zotero_json_fulltext/5500_zot2json.json'), WindowsPath('P:/Documents/research/1_sqlr_hierarchy/data/zotero_json_fulltext/5250_zot2json.json'), WindowsPath('P:/Documents/research/1_sqlr_hierarchy/data/zotero_json_fulltext/5000_zot2json.json'), WindowsPath('P:/Documents/research

AttributeError: 'str' object has no attribute 'str'

In [18]:
# put the looping over files into a copy of the previous code

from importlib.resources import path
import ijson
import re
import textwrap as tr
import pandas as pd

list_of_phrases = ["define hierarchy",
"defines hierarchy",
"defining hierarchy",
"definition for hierarchy",
"definition hierarchy",
"definition of hierarchy",
"hierarchy defined",
"hierarchy definition",
"hierarchy meaning",
"hierarchy refers",
"hierarchy they mean",
"hierarchy we mean",
"hierarchy i mean",
"define hierarchical",
"defines hierarchical",
"definition for hierarchical",
"definition hierarchical",
"definition of hierarchical",
"hierarchical defined",
"hierarchical definition",
"hierarchical i mean",
"hierarchical meaning",
"hierarchical they mean",
"hierarchical we mean",
"hierarchical refers"]

# How many characters on either side of the search phrase.
num_buffer = 280

# This function uses the lowercase for search, but then snips
# from fulltext that hasn't been made all lowercase so that the
# final snippets are more readable.

def get_snippets(sphrase, fulltextstr, lower_fulltextstr, fulltext_len):
    snip_array = []
    for occurrence in re.finditer(sphrase, lower_fulltextstr):
        if occurrence.start() != -1: # we don't have an else here, but could
            snip_start = occurrence.start() - num_buffer
            snip_end = occurrence.end() + num_buffer
            # print(sphrase, snip_start, snip_end)
            if snip_start < 0:
                snip_start = 0
            if snip_end > fulltext_len:
                snip_end = fulltext_len
            snippet = fulltextstr[snip_start:snip_end]
            # Make search phrase all caps to be easier to spot. Can be commented out.
            snippet = snippet.replace(sphrase, sphrase.upper()) 
            # print(snippet)
            snip_array.append(snippet)
    return(snip_array)

for jfile in zot2json_files:
    with open(jfile, 'rb') as input_file:
        parser = ijson.parse(input_file)
        review_array = []

        item_key = ''
        item_title = ''

        for prefix, event, value in parser:
            # TODO:
            # I'll need to get the key and titles here with ifs.
            # The value is different json info depending on the prefix.
            
            if (prefix, event) == ('item.key', 'string'):
                item_key = value # The Zotero key should be a unique ID

            if (prefix, event) == ('item.title', 'string'):
                item_title = value

            if (prefix, event) == ('item.fulltext', 'string'):
                # print('in the loop')
                nofi_fulltextstr = value.replace('ﬁ','fi')
                # clean_value = nofi_value
                # clean_value = re.sub(r'[_\W]+', ' ', value)
                lower_fulltextstr = nofi_fulltextstr.lower()
                # clean_value = re.sub(r'[_\W]+', ' ', value).lower().strip().replace('ﬁ','fi')
                # re.sub(r'[_\W]+', ' ', value).strip().lower()
                # if thelen == 107811: for testing
                fulltext_len = len(lower_fulltextstr)
                termcounter = 0
                for phrase in list_of_phrases:                
                    if phrase in lower_fulltextstr:
                        termcounter = termcounter + 1
                        # print((thelen, phrase))
                        snippet_array = get_snippets(phrase, nofi_fulltextstr, lower_fulltextstr, fulltext_len)
                        for snip in snippet_array:
                            review_array.append([item_key, item_title, phrase, snip])
                    # if thelen == 107811:
                    #     print(value)
                    #     print(" +++++++++++++++++++ STARTING STRIPPED STRING NOW +++++++++++++++++++++++++")
                    #     print(re.sub(r'[_\W]+', ' ', value).strip().lower())
            # if termcounter == 0: # and thelen > 400000 and thelen < 600000:                
            #     print("=========================== NEW ENTRY =================================")
            #     print(value)
                # print(" +++++++++++++++++++ STARTING STRIPPED STRING NOW +++++++++++++++++++++++++")
                # print(re.sub(r'[_\W]+', ' ', value).strip().lower())
        print(len(review_array))
        df = pd.DataFrame.from_dict(review_array, orient='columns')

        # Now construct the filename to use

        pstring = str(jfile)
        the_end = pstring.split(sep='\\')[-1]
        print(the_end)
        the_prefix = the_end.split(sep='_')[0]
        print('The prefix:', the_prefix)

        json_folder = Path("P:/Documents/research/1_sqlr_hierarchy/data/zotero_json_fulltext")
        new_fname = folder_name  / (str(the_prefix) + '_snippets.xlsx')
        print(new_fname)

        # Make the dataframe into an excel file with the same prefix as the json

        # df.to_csv(header=None, index=False, sep='\t', path_or_buf=new_fname).split('\n')
        df.to_excel(new_fname, index=False, header=True, engine='xlsxwriter')



# I can write this as a JSON file. Seemingly it's easy, one command.
# https://stackoverflow.com/a/56981871

# Instead, I'm going to write to a dataframe and then write that to
# a tab-separated CSV.

# Logic for writing a tab-separated CSV here.



105
6857_zot2json.json
The prefix: 6857
C:\Users\Rock\Downloads\6857_snippets.xlsx
230
6750_zot2json.json
The prefix: 6750
C:\Users\Rock\Downloads\6750_snippets.xlsx
207
6500_zot2json.json
The prefix: 6500
C:\Users\Rock\Downloads\6500_snippets.xlsx
246
6250_zot2json.json
The prefix: 6250
C:\Users\Rock\Downloads\6250_snippets.xlsx
245
6000_zot2json.json
The prefix: 6000
C:\Users\Rock\Downloads\6000_snippets.xlsx
302
5750_zot2json.json
The prefix: 5750
C:\Users\Rock\Downloads\5750_snippets.xlsx
250
5500_zot2json.json
The prefix: 5500
C:\Users\Rock\Downloads\5500_snippets.xlsx
215
5250_zot2json.json
The prefix: 5250
C:\Users\Rock\Downloads\5250_snippets.xlsx
246
5000_zot2json.json
The prefix: 5000
C:\Users\Rock\Downloads\5000_snippets.xlsx
189
4750_zot2json.json
The prefix: 4750
C:\Users\Rock\Downloads\4750_snippets.xlsx
233
4500_zot2json.json
The prefix: 4500
C:\Users\Rock\Downloads\4500_snippets.xlsx
218
4250_zot2json.json
The prefix: 4250
C:\Users\Rock\Downloads\4250_snippets.xlsx
200


### Testing code below

In [ ]:
# Make dataframe

import pandas as pd
df = pd.DataFrame.from_dict(review_array, orient='columns')

In [ ]:
# Dataframe can be stored as pickle

df.to_pickle("my_data.pkl")

Background: I have about 1.49 GB of JSON files, 50-80 KB each. They're JSON because I followed an example to output data from Zotero, and it used JSON. I realize that JSON is a serialization format, meaning I should be storing it, and reading and writing to it, in a different format; JSON is meant for communicating the data, not that sort of use.

Because of the large file size, I've used ijson to read the data incrementally and get what I need written to an array.

My issue is, now that I have the data, how should I store it for reading and writing, and how should I read and write to it?

How big would it be? I need to see how big the review array is. It's actually pretty small. 200 KB. All 28 files together would be about 5.7 MBs.

Ok, so how should I read and write to it? I guess the first thing to try would be a pandas table. Yep, easy.

But how do I store it on disk? The suggestion seems to be pickle.

If I pickle the df, how big is it? 132 KB. Even smaller.

df.to_pickle("my_data.pkl")
df = pd.read_pickle("my_data.pkl")

So I can just process each JSON, edit as a df, save as a pickle regularly.

I could also append just the coding changes to a temp file. I could do this for each "row" operation.

Ok, to summarize the design:

For the CLI UX:
The user specifies the JSON file to use after launching the app.
The user can specify to use the last file.
The user is looking at a blank definitional snippet.
The user needs to be able to go forward or back snippets.
    - JKL: keys and left-right arrow keys, larrow back and rarrow forward
The user needs to be able to code the currently displayed snippet with multiple codes (prob letters).
    - QWERTY keys fastest? WASD, Z, R, E, Q
The user needs to be able to blank the codes for the current displayed snippet.
    - Key in QWERTY maybe
The user needs to be able to go to the next blank entry
    - Down arrow key?

On the "backend" the app is:
1. Knows if there is a pickle file for the filename the user selects.
    - If so, dataframe is loaded from the pickle.
    - If not, creates a pickle file for that filename.
2. Knows the filename the user selected, and displays it. 
3. Knows the current key (df row number) of the snippet, which is just row number, and is unique for the file open.
4. Knows the item_key, which is not unique for the entry, but allows us to identify the exact bib item. (A bib item may have multiple definitional snippets.)
5. Knows the updates made to the data.
6. Writes each update made to the data, with a timestamp, to a history file (history.txt). 
    - filename, row number, item_key, search phrase, code
7. Writes each update made to the data to the dataframe.
8. Pickles the dataframe to file each time the user moves from one snippet to another, IF something has changed.
9. Knows the number of blank codings, which the CLI displays to the user.

Outline the functions necessary.
Write the logic for all the manipulations, but don't require key presses.
Each of these functions will be triggered by a key press.

On launch,
App checks to see if the directory it's in has any JSON files that match "####_zot2json.json".
    - If not, it says "No zot2json.json files in this dir. Maybe you need to run this py in another dir?"

file_selection_menu()
App displays two options with one input: (maybe valid_file needs to be true, and it starts false?)
    1. Check for history.txt. 
        If none found, set last_pickle_edited to ''. 
        If one found, set last_pickle_edited to that filename, print "Most recent file is " + last_pickle_edited.
    1. "Specify json file to load using number prefix here, or enter L for (l)ast coding file in history:"
        - valid input is ### or #### numbers, or L.
        - if invalid input, say "Invalid. Enter a 3- or 4-digit number for _zot2json.json file or (l)ast. Hit Enter."
            - Loop back to file_selection_menu.
        - if ### or #### number is entered, check if that number + _zot2json.json is found in dir.
            - If not found, display 'That file isn't in this dir.' Loop back to file_selection_menu.
        - if L entered, look at the last line of the history.txt for the pickle filename
            if it exists, load it and break this loop.
            if it doesn't exist, say "No history.txt file detected."


def check_history_last_file():  # This may not be important enough to write.
    last_pickle_edited = ''
    if history.txt is in the dir:
        with open file for reading as whatever:
            get filename from last line
            last_pickle_edited = that filename
    return last_pickle_edited

def check_filenumberprefix_valid(userinput):
    if userinput not (check regex for ddd or dddd):
        return('')
    else:
        check_jsonfile_in_dir(userinput)

...but these are important enough to write

   

Minimum viable:
1. Loading menu / file selection menu. Loads the json snippet file. (not zot2json, I need to remember these are snippet files)
2. Coding menu. Appears only when a file is loaded. Once a file is loaded:
    1. Reads and tells how many of the rows have a blank for the coding. Says "x/ttlrows coded."
    2. Displays the snippet text for a random blank entry.
    3. Shows coding options (letters and what they mean).
    4. Shows navigation options.

Loading menu
coding main menu
coding options display
navigation options display
blank fraction display

Code to randomly shuffle a list:
import random
l1=['aa',22,'ff',15,90,5.55]
random.shuffle(l1)

I can use a pointer for the position of the menu in the list.

def file_selection_menu():
    if not check_dir_for_json():
        print("No snippet.json files in this dir. Place this py in different dir?")
    else:

def check_dir_for_json():
    if "_zot2json.json" in file_list:
        return true

def check_df_blank_codings(active_filename):
    check for associated pickle file

    <!-- last_filename = check_history_last_file()
    if last_filename:
        print('Last file worked on: ' + last_filename) -->



### Object Oriented Programming style?

I think I could probably make a menu system using OOP style.

But I am not sure how. Maybe some MVC approach.
Like https://www.giacomodebidda.com/posts/mvc-pattern-in-python-introduction-and-basicmodel/


In [ ]:

import sys

file_menu_options = {
    'll': '(ll)oad a file',
    'pp': 'exit, (pp)eace out',
}

def print_file_menu():
    for key in file_menu_options.keys():
        print (key, '--', file_menu_options[key] )

nav_menu_options = {
    'j': 'back to last item',
    'k': 'forward to next blank',
}

def print_nav_menu():
    for key in nav_menu_options.keys():
        print (key, '--', nav_menu_options[key] )


# I'll need a column for each in the df

coding_menu_options = {
    'w': 'rank',
    'a': 'nest',
    'd': 'control',
    'q': 'non-definition, the content is not really defining hierarchy',
    'e': 'other, a legit definition that is outside of rank, nest, control definitions',
    's': 'non-social, definition not related to social hierarchies',
    0: 'go to file menu'
}

def print_coding_menu():
    for key in coding_menu_options.keys():
        print (key, '--', coding_menu_options[key] )

def print_blank_fraction():
    blanks_count = 5
    total_entries = 10
    print('Completed: ', blanks_count, '/', total_entries)       

def file_path_solicitation():
    entered_filename = ''
    while not entered_filename:
        try:
            entered_filename = int(input('Please enter the first digits of the JSON file path: '))
        except:
            print('Invalid input. Please enter a number...')
    if entered_filename:
        print('Thank you for entering filename', entered_filename)
    return entered_filename

# This currently can't handle them entering an option when there's
# no filename. Even if the menu option is invisible, they could still enter
# w or s or somethign without an active file.

def execute_option(option):
    print('Option executor reached...') # Legit option, go on...
    print('Test nonlocal jsonfilepath is:', jsonfilepath)
    if option in file_menu_options:
        print('I will save coding to the df') # moving entries, save the data to the df first
        print('Current json file is', jsonfilepath)
        if option == 'pp':
            print('Saving work to dataframe...')
            print('Here I would actually be saving instead of printing.')
            print('Exiting.')
            sys.exit()
        if option == 'll': # '(ll)oad a file',
            jsonfilepath = file_path_solicitation()
            print('In theory I just loaded', jsonfilepath)
    
    elif option in nav_menu_options and jsonfilepath:
        print('Saving work to dataframe...')
        print('Here I would actually be saving instead of printing.')
        
        if option == 'j': # 'back to last item',
            print('back to last item')
        if option == 'k': # 'forward to next blank',
            print('forward to next blank')
        if option == 0: # 'go to file menu'
            print('go to file menu')

    elif option in coding_menu_options and jsonfilepath:   
        if option == 'w': # 'rank',
            print('rank')
        if option == 'a': # 'nest',
            print('nest')
        if option == 'd': # 'control',
            print('control')
        if option == 'q': # 'non-definition, the content is not really defining hierarchy',
            print('non-definition, the content is not really defining hierarchy')
        if option == 'e': # 'other, a legit definition that is outside of rank, nest, control definitions',
            print('other, a legit definition that is outside of rank, nest, control definitions')
        if option == 's': # 'non-social, definition not related to social hierarchies',
            print('non-social, definition not related to social hierarchies')
   
    else:
        print('Invalid option. Please use option from menu.')

if __name__=='__main__':
    
    jsonfilepath = ''
    
    while(True):
        
        option = ''
        
        print_file_menu()

        print('Printing jsonfilepath:', jsonfilepath) # TODO here is the issue.

        if jsonfilepath:
            print('You loaded file so now you can access the whole menu.')
            print_coding_menu()
            print_nav_menu()
            print_blank_fraction()        
        
        try:
            option = input('Enter your choice: ')
        except:
            print('Invalid input. Please revise...')

        execute_option(option)




### Text for making a stackoverflow question on the issues I had with var scopes in making CLI
I was putting together a small CLI program to enable me to view and manually label text blurbs I would load in from a json file. However, I've run into problems with variable scope and the while loop of the menu. The intent is to launch the program, then specify a prefix to load a particular file, and then have a menu that enables me to load a different file while within the program.



In [10]:
# minimum viable example for posting on stack exchange.
# issues with variable scope, functions, and making a CLI menu

# Errors/issues:
# Nonlocal declaration not allowed at module level
# No binding for nonlocal "jsonfilepath" found

import sys

file_menu_options = {
    'l': '(l)oad a file',
    'q': '(q)uit',
}

def print_file_menu():
    for key in file_menu_options.keys():
        print (key, ': ', file_menu_options[key] )

def file_path_solicitation():
    filename = ''
    while not filename:
        try:
            filename = int(input('Please enter the first digits of the JSON file path: '))
        except:
            print('Invalid input. Please enter a number...')
    if filename:
        print('Thank you for entering filename ' + str(filename) + 'filestem.json')
    return filename

def execute_option(option):
    if option in file_menu_options:
        if option == 'q': # 'q': '(q)uit'
            print('Pretending to save work to file...')
            print('Exiting.')
            sys.exit()
        if option == 'l': # '(l)oad a file'
            # nonlocal jsonfilepath # <- !! No binding for nonlocal "jsonfilepath" found
            jsonfilepath = file_path_solicitation()
            print('In theory I just loaded', jsonfilepath)
    else:
        print('Invalid option. Please use option from menu.')

if __name__=='__main__':
    jsonfilepath = ''
    while(True):
        option = ''
        # nonlocal jsonfilepath # <- !! Nonlocal declaration not allowed at module level
        print_file_menu()
        print('Printing jsonfilepath:', jsonfilepath) # Always blank.

        if jsonfilepath:
            print('You loaded file so now you can access the whole menu.')
            print('A B C D here are some fake menu options.')
        
        try:
            option = input('Enter your choice: ')
        except:
            print('Invalid input. Please revise...')

        execute_option(option)

l :  (l)oad a file
q :  (q)uit
Printing jsonfilepath: 
Thank you for entering filename 1234filestem.json
In theory I just loaded 1234
l :  (l)oad a file
q :  (q)uit
Printing jsonfilepath: 
Pretending to save work to file...
Exiting.
l :  (l)oad a file
q :  (q)uit
Printing jsonfilepath: 
Pretending to save work to file...
Exiting.
l :  (l)oad a file
q :  (q)uit
Printing jsonfilepath: 


In [148]:
# CLI menu code from 
# https://computinglearner.com/how-to-create-a-menu-for-a-python-console-application/

menu_options = {
    1: 'Option 1',
    2: 'Option 2',
    3: 'Option 3',
    4: 'Exit',
}

def print_menu():
    for key in menu_options.keys():
        print (key, '--', menu_options[key] )

def option1():
     print('Handle option \'Option 1\'')

def option2():
     print('Handle option \'Option 2\'')

def option3():
     print('Handle option \'Option 3\'')

if __name__=='__main__':
    while(True):
        print_menu()
        option = ''
        try:
            option = int(input('Enter your choice: '))
        except:
            print('Wrong input. Please enter a number ...')
        #Check what choice was entered and act accordingly
        if option == 1:
           option1()
        elif option == 2:
            option2()
        elif option == 3:
            option3()
        elif option == 4:
            print('Thanks message before exiting')
            exit()
        else:
            print('Invalid option. Please enter a number between 1 and 4.')

1 -- Option 1
2 -- Option 2
3 -- Option 3
4 -- Exit
Handle option 'Option 1'
1 -- Option 1
2 -- Option 2
3 -- Option 3
4 -- Exit
Wrong input. Please enter a number ...
Invalid option. Please enter a number between 1 and 4.
1 -- Option 1
2 -- Option 2
3 -- Option 3
4 -- Exit
Wrong input. Please enter a number ...
Invalid option. Please enter a number between 1 and 4.
1 -- Option 1
2 -- Option 2
3 -- Option 3
4 -- Exit


In [146]:
import gc
import sys

def get_obj_size(obj):
    marked = {id(obj)}
    obj_q = [obj]
    sz = 0

    while obj_q:
        sz += sum(map(sys.getsizeof, obj_q))

        # Lookup all the object referred to by the object in obj_q.
        # See: https://docs.python.org/3.7/library/gc.html#gc.get_referents
        all_refr = ((id(o), o) for o in gc.get_referents(*obj_q))

        # Filter object that are already marked.
        # Using dict notation will prevent repeated objects.
        new_refr = {o_id: o for o_id, o in all_refr if o_id not in marked and not isinstance(o, type)}

        # The new obj_q will be the ones that were not marked,
        # and we will update marked with their ids so we will
        # not traverse them again.
        obj_q = new_refr.values()
        marked.update(new_refr.keys())

    return sz

print('Size: ' + str(get_obj_size(review_array)*0.001) + ' KB.')
# So with 28 JSON files, if they're all roughly like this, that's 5.7 MBs.

Size: 204.958 KB.


In [144]:
# Make dataframe

import pandas as pd
df = pd.DataFrame.from_dict(review_array, orient='columns')

In [147]:
df.to_pickle("my_data.pkl")

In [139]:
# TEST REVIEW ARRAY PRINTER

for i in review_array:
    print(i[0:3])
    blurb = i[3]
    blurb = tr.wrap(blurb, width=80)
    for b in blurb:
        print(b)
    print('++')
    print(i)
    print('--')

['2ZIFHLZ8', 'Political Compassions under Pandemic Spectacles', 'hierarchical definition']
esolved by granting Indonesia the “final say” when it came to the
commercialization of drugs developed based on Indonesian data. The Indonesian
government’s actions offered a rare glimpse into the deeply political nature of
the Western pandemic spectacles and into the underlying HIERARCHICAL DEFINITION
of human polity. The controversy over sharing data illuminates the politics of
health: alternative visions, different agendas, co-optive purposes, and clashing
interests. It differentiates among actors and defines the way in which they
collaborate. Even the modern expert-driven
++
['2ZIFHLZ8', 'Political Compassions under Pandemic Spectacles', 'hierarchical definition', 'esolved by granting Indonesia the “final say” when it came to the commercialization of drugs developed based on Indonesian data. The Indonesian government’s actions offered a rare glimpse into the deeply political nature of the Wes

In [137]:
## JSON-MAKER FOR REVIEW DATA

# 1. Make python dictionary from review_array
# 2. Write that dictionary to a json file using 
# print(json.dumps(my_dictionary, indent=4, sort_keys=True))

import json

newkeys = ['item_key', 'item_title', 'search_phrase', 'snip', 'hier_type_code']

json_for_reviewing = []

for row in review_array:
    row = row + ['']
    dictrow = {key:value for (key, value) in zip(newkeys, row)}
    json_for_reviewing.append(dictrow)
# list = [{"pack": x[0], "pack": x[1], "price": x[2], "checkstate": x[3]} for x in foodata]


foo = json.dumps(json_for_reviewing, indent=4, sort_keys=False)
print(foo)

[
    {
        "item_key": "I6F9UF2D",
        "item_title": "Literature-based discovery for oceanographic climate science",
        "search_phrase": "hierarchy defined",
        "snip": "ow a larger context can be exploited to yield higher performance for nouns.\nIn an LBD context, the learning process can be seen as bootstrapping a set of concepts for the domain. The resulting system can be considered a hybrid between a type 1 and type 2 approach in terms of the HIERARCHY DEFINED in 2.4, as it does not use any domain knowledge, but still proposes a set of concepts. A hypothesis that will be evaluated empirically is whether this will provide better results than a pure type 1 system.\nThe ontology learned by the system can be edited by a domain expert, or c",
        "hier_type_code": ""
    },
    {
        "item_key": "YLBTITKB",
        "item_title": "Us and them: Black-White relations in the wake of Hispanic population growth",
        "search_phrase": "hierarchy defined",
       

In [ ]:
print(review_array[0:4])

### Testing code below

In [136]:

# 1. Make python dictionary from review_array
# 2. Write that dictionary to a json file using 
# print(json.dumps(my_dictionary, indent=4, sort_keys=True))

import json

somekeys = ['item_key', 'item_title', 'search_phrase', 'snip', 'hier_type_code']
somedata = [['231V2V41', 'Entitled to being a title', 'search phrase', 'here I am a short little snippet'],
           ['I6F9UF2D', 'Oceanographic climate science', 'climate change', 'Lorem ipsum dolor sit amet']]

json_input = []

for arow in somedata:
    arow = arow + ['']
    dictrow = {key:value for (key, value) in zip(somekeys, arow)}
    json_input.append(dictrow)

# The zip() function returns a zip object, which is an iterator of tuples where 
# the first item in each passed iterator is paired together, and then the second 
# item in each passed iterator are paired together etc. -w3schools

foo = json.dumps(json_input, indent=4, sort_keys=False)
print(foo)



[
    {
        "item_key": "231V2V41",
        "item_title": "Entitled to being a title",
        "search_phrase": "search phrase",
        "snip": "here I am a short little snippet",
        "hier_type_code": ""
    },
    {
        "item_key": "I6F9UF2D",
        "item_title": "Oceanographic climate science",
        "search_phrase": "climate change",
        "snip": "Lorem ipsum dolor sit amet",
        "hier_type_code": ""
    }
]


In [79]:
gobble1 = '''define hierarchy VandeHei said he wanted Axios to be a "mix between The Economist and Twitter".[3] The company initially covered a mix of business, politics, technology, 
health care, and media. VandeHei said Axios would focus on the "collision between tech and areas such as bureaucracy, healthcare, energy, and the transportation 
infrastructure".[2] At launch, Nicholas Johnston, a former managing editor at Bloomberg L.P., was named editor-in-chief.[4]'''
gobble2 = '''In the summer of 2016, Axios secured $10 million in a round of financing led by Lerer Hippeau Ventures. Backers include media-partner NBC News, 
Laurene Powell Jobs' Emerson Collective, Greycroft Partners, and David and Katherine Bradley, owners of Atlantic Media.[2] The company had raised $30 million as 
of November 2017.[5][6] It planned to focus on "business, technology, politics, and media trends".[2] Axios generates revenue through short-form native 
advertising and sponsored newsletters.[7] It earned more than $10 million in revenue in its first seven months.[5] Its advertisers include Exxon Mobil and 
Koch Industries.[undue weight? discuss][1] define hierarchy yee haw'''

aphrase = 'define hierarchy'
stub = "island’s hierarchy—meaning her sole. deﬁne hierarchy. hierarchy. Deﬁnition and here we define hierarchy again, 2nd time"
sentence = gobble1 + ' ' + stub + ' ' + gobble2
sentence = re.sub(r'[_\W]+', ' ', sentence).lower().strip().replace('ﬁ','fi')
sentence.find(aphrase)
# This will give you the start position of the word (if it exists, otherwise -1), 
# then you can just add the length of the word to it in order to get the index of its end.

start_index = sentence.find(aphrase)
if start_index == -1:
    print("NOT FOUND")
end_index = start_index + len(aphrase) # if the start_index is not -1
print(start_index, " ", end_index)

# Now let's do multiple occurrences

import re

num_buffer = 280

# for occurrence in re.finditer(aphrase, sentence):
#     print(aphrase, occurrence.start(), occurrence.end())
#     print(sentence[occurrence.start()-num_buffer:occurrence.end()+num_buffer])

sen_len = len(sentence)

for occurrence in re.finditer(aphrase, sentence):
    if occurrence.start() != -1:
        snip_start = occurrence.start() - num_buffer
        snip_end = occurrence.end() + num_buffer
        print(aphrase, snip_start, snip_end)
        if snip_start < 0:
            snip_start = 0
        if snip_end > sen_len:
            snip_end = sen_len
        snippet = sentence[snip_start:snip_end].replace(aphrase, aphrase.upper()) 
        print(snippet)
        # return(snippet)


0   16
define hierarchy -280 296
DEFINE HIERARCHY vandehei said he wanted axios to be a mix between the economist and twitter 3 the company initially covered a mix of business politics technology health care and media vandehei said axios would focus on the collision between tech and areas such as bureaucracy healthcare energy a
define hierarchy 189 765
ehei said axios would focus on the collision between tech and areas such as bureaucracy healthcare energy and the transportation infrastructure 2 at launch nicholas johnston a former managing editor at bloomberg l p was named editor in chief 4 island s hierarchy meaning her sole DEFINE HIERARCHY hierarchy definition and here we DEFINE HIERARCHY again 2nd time in the summer of 2016 axios secured 10 million in a round of financing led by lerer hippeau ventures backers include media partner nbc news laurene powell jobs emerson collective greycroft partners and david and ka
define hierarchy 239 815
en tech and areas such as bureaucracy healt

In [ ]:
# ASSESSING SEARCH TERMS

# We should find at least one search term per item.
# If we find less, that means a discrepancy between this script
# and google scholar. Those need to be found and minimized.

# Now, make it so that it counts how many times 
# for (def phrase) in fulltext:

# for fulltext:
#   if phrase in fulltext for phrase in list_of_phrases:
#        print(phrase)
# word in sentence for word in listOfWords

import ijson
import re

list_of_phrases = ["define hierarchy",
"defines hierarchy",
"defining hierarchy",
"definition for hierarchy",
"definition hierarchy",
"definition of hierarchy",
"hierarchy defined",
"hierarchy definition",
"hierarchy meaning",
"hierarchy refers",
"hierarchy they mean",
"hierarchy we mean",
"hierarchy i mean",
"define hierarchical",
"defines hierarchical",
"definition for hierarchical",
"definition hierarchical",
"definition of hierarchical",
"hierarchical defined",
"hierarchical definition",
"hierarchical i mean",
"hierarchical meaning",
"hierarchical they mean",
"hierarchical we mean",
"hierarchical refers"]

lenarray = []


with open(json_file, 'rb') as input_file:
    parser = ijson.parse(input_file)
    for prefix, event, value in parser:
        if (prefix, event) == ('item.fulltext', 'string'):
            # print('in the loop')
            thelen = len(value)
            nofi_value = value.replace('ﬁ','fi')
            # clean_value = nofi_value
            # clean_value = re.sub(r'[_\W]+', ' ', value)
            clean_value = nofi_value.lower()
            # clean_value = re.sub(r'[_\W]+', ' ', value).lower().strip().replace('ﬁ','fi')
            # re.sub(r'[_\W]+', ' ', value).strip().lower()
            # if thelen == 107811: for testing
            termcounter = 0
            for phrase in list_of_phrases:                
                if phrase in clean_value:
                    termcounter = termcounter + 1
                    # print((thelen, phrase))
                    lenarray.append((thelen, phrase))
                    # if thelen == 107811:
                    #     print(value)
                    #     print(" +++++++++++++++++++ STARTING STRIPPED STRING NOW +++++++++++++++++++++++++")
                    #     print(re.sub(r'[_\W]+', ' ', value).strip().lower())
            # if termcounter == 0: # and thelen > 400000 and thelen < 600000:                
            #     print("=========================== NEW ENTRY =================================")
            #     print(value)
                # print(" +++++++++++++++++++ STARTING STRIPPED STRING NOW +++++++++++++++++++++++++")
                # print(re.sub(r'[_\W]+', ' ', value).strip().lower())

                



print(len(lenarray))
# # print(lenarray)
# for q,w in lenarray:
#   print(f"{q}\t{w}")

## No-match entries that were matches in google scholar
# These must have character issues or punctuation.

# In one instance, the occurrence was "island’s hierarchy—meaning her sole"
# I think this is an emdash, could be an endash
# "deﬁne hierarchy" has the combo fi character
# "deﬁnes hierarchy" has that combo
# "the hierarchy (meaning more easy to identify referents) "
# "hierarchy. Deﬁnition 7 Output" has the combo fi again
# "hierarchy deﬁned by Nadeau and Sekine", which occured 1 more time
# "Hierarchy. Refers"
# "formal hierarchy, meaning an officially"


# DONE:
# 1. the fi combo character
# 2. strip all punctuation and special chars like (). Didnt' do this on purpose.

# DONE: 
# 1. Create parsing code that takes the embedded sentence and one on either side.
#   - Difficult because of instances like "Hierarchy. Refers"
#   - However, it might be possible to get the relative positions of those characters

# DONE:
#   - consider that it might not be worth it to strip everything out and lower case everthing.
#   - How can I keep the sentence as readable as possible?
#   - I could make things lowercase ONLY, leave all punctuation.
#   - I need to run everything through and begin making the arrays. See how readable different types are.

## Dealing with case.
# Even if I search all lower, that alone shouldn't change the position of words.
# Thus I can use the lower position in the original string and still get the word.

# The types...
# 1. Replace the fi char with fi. That should happen in ALL cases.
# Results? 160.

# 2. Clean all white space?
# Results? 170

# 3. Search all lower case.
# Results? 134

# 4. No-fi and lower.
# Results? 173

In [51]:
s = "island’s hierarchy—meaning her sole. deﬁne hierarchy. hierarchy. Deﬁnition"
re.sub(r'[_\W]+', ' ', s).lower().strip().replace('ﬁ','fi')


'island s hierarchy meaning her sole define hierarchy hierarchy definition'

In [24]:
# sentence = "As far as what defines hierarchy, or what hierarchy refers to, scholars employ many meanings."

# if (phrase in sentence for phrase in list_of_phrases):
#     print(phrase)


# list_of_phrases = ["defines power", "power refers"]

# sentence = "As far as what defines power, or what power refers to, people employ many meanings."

# if (phrase in sentence for phrase in list_of_phrases):
#     print(phrase)

for phrase in list_of_phrases:
    if phrase in sentence:
        print(phrase)

defines power
power refers
